# Практическое задание

## Данные о студенте

1. **ФИО**: Денисов Даниил Михайлович
2. **Факультет**: Механико-математический
3. **Курс**: 2 (магистратура)
4. **Группа**: М2

## Замечания

* Заполненный ноутбук необходимо сдать боту
* Соблюдаем кодекс чести (по нулям и списавшему, и давшему списать)
* Можно (и нужно!) применять для реализации только библиотеку **Numpy**
* Ничего, крому Numpy, нельзя использовать для реализации 
* **Keras** используется только для тестирования Вашей реализации
* Если какой-то из классов не проходит приведенные тесты, то соответствующее задание не оценивается
* Возможно использование дополнительных (приватных) тестов
 

## Реализация собственного нейросетевого пакета для запуска и обучения нейронных сетей

Задание состоит из трёх частей:
1. Реализация прямого вывода нейронной сети (5 баллов)
2. Реализация градиентов по входу и распространения градиента по сети (5 баллов)
3. Реализация градиентов по параметрам и метода обратного распространения ошибки с обновлением парметров сети (10 баллов)

Дополнительные баллы можно получить при реализации обучения сети со свёрточными слоями (10 баллов), с транспонированной свёрткой (10 баллов), дополнительного оптимизатора (5 баллов). 

###  1. Реализация вывода собственной нейронной сети

1.1 Внимательно ознакомьтесь с интерфейсом слоя. Любой слой должен содержать как минимум три метода:
- конструктор
- прямой вывод 
- обратный вывод, производные по входу и по параметрам

In [1]:
class Layer(object):
    def __init__(self):
        self.name = 'Layer'
    
    def forward(self, input_data):
        pass

    def backward(self, input_data):
        return [self.grad_x(input_data), self.grad_param(input_data)]
    
    def grad_x(self, input_data):
        pass

    def grad_param(self, input_data):
        return []
    
    def update_param(self, grads, learning_rate):
        pass

1.2 Ниже предствален интерфейс класса  Network. Обратите внимание на реализацию метода predict, который последовательно обрабатывает входные данные слой за слоем.

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

class Network(object):
    def __init__(self, layers, loss=None):
        self.name = 'Network'
        self.layers = layers
        self.loss = loss
    
    def forward(self, input_data):
        return self.predict(input_data)
    
    def grad_x(self, input_data, labels):
        # Intermediary gradients (forward pass)
        grads_inter = []
        current_input = input_data
        for layer in self.layers:
            grads_inter.append(layer.grad_x(current_input))
            current_input = layer.forward(current_input)

        # Target gradient (backward pass)
        grad = self.loss.grad_x(current_input, labels)
        for layer_grad in reversed(grads_inter):
            grad = np.einsum('bi,bij->bj', grad, layer_grad)

        return grad
    
    def grad_param(self, input_data, labels):
        # Intermediary gradients (forward pass)
        grads_inter = []
        current_input = input_data
        for layer in self.layers:
            grads_inter.append(layer.backward(current_input))
            current_input = layer.forward(current_input)
        
        # Target gradients (backward pass)
        grads_targ = []
        grad = self.loss.grad_x(current_input, labels)
        for layer_grad_x, layer_grad_param in reversed(grads_inter):
            grads_targ.append([np.einsum('bi,bij->bj', grad, grad_param) 
                               for grad_param in layer_grad_param])
            grad = np.einsum('bi,bij->bj', grad, layer_grad_x)

        return grads_targ[::-1]

    def update(self, grads, learning_rate):
        for layer, grad in zip(self.layers, grads):
            layer.update_param(grad, learning_rate)
    
    def predict(self, input_data):
        current_input = input_data
        for layer in self.layers:
            current_input = layer.forward(current_input)
        return current_input
    
    def calculate_loss(self, input_data, labels):
        return self.loss.forward(self.predict(input_data), labels)
    
    def train_step(self, input_data, labels, learning_rate=0.001):
        grads = self.grad_param(input_data, labels)
        self.update(grads, learning_rate)
    
    def fit(self, trainX, trainY, validation_split=0.25, 
            batch_size=1, nb_epoch=1, learning_rate=0.01):
        
        train_x, val_x, train_y, val_y = train_test_split(trainX, trainY, 
                                                          test_size=validation_split,
                                                          random_state=42)
        
        for epoch in range(nb_epoch):
            for i in tqdm(range(int(len(train_x)/batch_size))):
                batch_x = train_x[i*batch_size: (i+1)*batch_size]
                batch_y = train_y[i*batch_size: (i+1)*batch_size]
                self.train_step(batch_x, batch_y, learning_rate)
            
            print('%d epoch: val %.2f' %(epoch + 1, self.evaluate(val_x, val_y)))
            
    def evaluate(self, testX, testY):
        y_pred = np.argmax(self.predict(testX), axis=1)
        y_true = np.argmax(testY, axis=1)
        return np.sum(y_pred == y_true) / len(y_true)

#### 1.1 Необходимо реализовать метод forward для вычисления следующих слоёв:

- DenseLayer
- ReLU
- Softmax
- FlattenLayer
- MaxPooling

In [1]:
import numpy as np

In [4]:
class DenseLayer(Layer):
    def __init__(self, input_dim, output_dim, W_init=None, b_init=None):
        self.name = 'Dense'
        self.input_dim = input_dim
        self.output_dim = output_dim

        # Use LeCun initialization by default
        self.W = W_init if W_init is not None \
                 else np.sqrt(3 / input_dim) * (2 * np.random.random((input_dim, output_dim)) - 1)
        self.b = b_init if b_init is not None \
                 else np.zeros(output_dim, 'float32')
    
    def forward(self, input_data):
        return np.einsum('bi,ij->bj', input_data, self.W) + self.b

    def grad_x(self, input_data):
        # dy/dx = W^T
        return np.tile(self.W.T, 
                       reps=(input_data.shape[0], 1, 1))

    def grad_b(self, input_data):
        # dy/db = I
        return np.tile(np.eye(len(self.b)), 
                       reps=(input_data.shape[0], 1, 1))

    def grad_W(self, input_data):
        # dy/dW = (x_1 * I ... x_n * I)
        W_rows, W_cols = self.W.shape
        mats = np.tile(np.eye(W_cols), 
                       reps=(input_data.shape[0], W_rows, 1, 1))
        mats *= np.expand_dims(input_data, axis=(2, 3))
        return np.concatenate(np.transpose(mats, axes=(1, 0, 2, 3)), axis=2)
    
    def update_W(self, grad, learning_rate):
        self.W -= learning_rate * np.mean(grad, axis=0).reshape(self.W.shape)
    
    def update_b(self, grad, learning_rate):
        self.b -= learning_rate * np.mean(grad, axis=0)
        
    def update_param(self, params_grad, learning_rate):
        self.update_W(params_grad[0], learning_rate)
        self.update_b(params_grad[1], learning_rate)
    
    def grad_param(self, input_data):
        return [self.grad_W(input_data), self.grad_b(input_data)]

In [5]:
class ReLU(Layer):
    def __init__(self):
        self.name = 'ReLU'
    
    def forward(self, input_data):
        return np.clip(input_data, 0, None)

    def grad_x(self, input_data):
        res = input_data.copy()
        res[res < 0] = 0
        res[res > 0] = 1
        return np.expand_dims(res, axis=1) * np.expand_dims(np.eye(res.shape[1]), axis=0)

In [6]:
class Softmax(Layer):
    def __init__(self):
        self.name = 'Softmax'
    
    def forward(self, input_data):
        exps = np.e ** input_data
        return exps / np.sum(exps, axis=1, keepdims=True)
    
    def grad_x(self, input_data):
        # dy/dx = diag(e^x/S) - (e^x/S)^T * (e^x/S)
        forward = self.forward(input_data)
        res = np.expand_dims(forward, axis=1) * np.expand_dims(np.eye(forward.shape[1]), axis=0)
        res -= np.expand_dims(forward, axis=2) @ np.expand_dims(forward, axis=1)
        return res

In [7]:
class FlattenLayer(Layer):
    def __init__(self):
        self.name = 'Flatten'
        
    def forward(self, input_data):
        return input_data.reshape(input_data.shape[0], -1)
    
    def grad_x(self, input_data):
        return np.eye(input_data[0].size)

#### 1.2 Реализуйте теперь свёрточный слой и транспонированную свёртку  (опционально)

In [167]:
b, c_in, c_out, h, w = 2, 2, 1, 4, 4
img1 = np.arange(h*w).reshape((h, w))
img = np.array([[img1, img1 + 1], [img1 + 2, img1 + 3]], dtype='float32')
img

array([[[[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [12., 13., 14., 15.]],

        [[ 1.,  2.,  3.,  4.],
         [ 5.,  6.,  7.,  8.],
         [ 9., 10., 11., 12.],
         [13., 14., 15., 16.]]],


       [[[ 2.,  3.,  4.,  5.],
         [ 6.,  7.,  8.,  9.],
         [10., 11., 12., 13.],
         [14., 15., 16., 17.]],

        [[ 3.,  4.,  5.,  6.],
         [ 7.,  8.,  9., 10.],
         [11., 12., 13., 14.],
         [15., 16., 17., 18.]]]], dtype=float32)

In [199]:
# Convolution
k, s = 2, 2
ker = np.tile(np.eye(k), reps=(c_in, 1, 1)) * np.array([[[1]], [[2]]])
ker = np.tile(ker, reps=(c_out, 1, 1, 1))
ker = ker.transpose(2, 3, 1, 0)
ker

array([[[[1.],
         [2.]],

        [[0.],
         [0.]]],


       [[[0.],
         [0.]],

        [[1.],
         [2.]]]])

In [178]:
h_out, w_out = (h - k) // s + 1, (w - k) // s + 1
h_out, w_out

(2, 2)

In [185]:
idh = np.arange(k) + np.expand_dims(s * np.arange(h_out), axis=1).repeat(k, axis=1)
idh = np.expand_dims(idh.repeat(w_out, axis=0), axis=2)
idh

array([[[0],
        [1]],

       [[0],
        [1]],

       [[2],
        [3]],

       [[2],
        [3]]])

In [186]:
idw = np.arange(k) + (s * np.arange(w_out))[:, None].repeat(k, axis=1)
idw = np.expand_dims(np.tile(idw, reps=(h_out, 1)), axis=1)
idw

array([[[0, 1]],

       [[2, 3]],

       [[0, 1]],

       [[2, 3]]])

In [200]:
patches = img[:, :, idh, idw]
patches

array([[[[[ 0.,  1.],
          [ 4.,  5.]],

         [[ 2.,  3.],
          [ 6.,  7.]],

         [[ 8.,  9.],
          [12., 13.]],

         [[10., 11.],
          [14., 15.]]],


        [[[ 1.,  2.],
          [ 5.,  6.]],

         [[ 3.,  4.],
          [ 7.,  8.]],

         [[ 9., 10.],
          [13., 14.]],

         [[11., 12.],
          [15., 16.]]]],



       [[[[ 2.,  3.],
          [ 6.,  7.]],

         [[ 4.,  5.],
          [ 8.,  9.]],

         [[10., 11.],
          [14., 15.]],

         [[12., 13.],
          [16., 17.]]],


        [[[ 3.,  4.],
          [ 7.,  8.]],

         [[ 5.,  6.],
          [ 9., 10.]],

         [[11., 12.],
          [15., 16.]],

         [[13., 14.],
          [17., 18.]]]]], dtype=float32)

In [203]:
bias = np.array([1])
conv = np.tensordot(patches, ker, axes=([1, 3, 4], [2, 0, 1]))
conv = conv.reshape(b, c_out, h_out, w_out) + np.expand_dims(bias, axis=(0, 2, 3))
conv

array([[[[20., 32.],
         [68., 80.]]],


       [[[32., 44.],
         [80., 92.]]]])

In [ ]:
class Conv2DLayer(Layer):
    def __init__(self, kernel_size=3, input_channels=2, output_channels=3, 
                 padding='same', stride=1, K_init=None, b_init=None):
        # padding: 'same' или 'valid'
        # Работаем с квадратными ядрами, поэтому kernel_size - одно число
        # Работаем с единообразным сдвигом, поэтому stride - одно число
        # Фильтр размерности [kernel_size, kernel_size, input_channels, output_channels]
        self.name = 'Conv2D'
        self.kernel_size = kernel_size
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.kernel = K_init if K_init is not None \
                      else np.random.random((kernel_size, kernel_size, 
                                             input_channels, output_channels))
        self.bias = b_init if b_init is not None \
                    else np.zeros(output_channels)
        self.padding = padding
        self.stride = stride
    
    def forward(self, input_data):
        # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
        # Вначале нужно проверить на согласование размерностей входных данных и ядра!
        # Нужно заполнить Numpy-тензор out
        b, c_in, h_in, w_in = input_data.shape
        if c_in != self.input_channels:
            raise ValueError("Input channels mismatch.")
        if h_in < self.kernel_size or w_in < self.kernel_size:
            raise ValueError("Dimensions mismatch.")
        
        k, s, c_out = self.kernel_size, self.stride, self.output_channels
        if self.padding == 'same':
            h_out, w_out = h_in, w_in
            h_pad = np.ceil((k + s * (h_in - 1) - h_in) / 2)
            w_pad = np.ceil((k + s * (w_in - 1) - w_in) / 2)
            source = np.pad(input_data, ((0,), (0,), (h_pad,), (w_pad,)))
        elif self.padding == 'valid':
            h_out = (h_in - k) // s + 1
            w_out = (w_in - k) // s + 1
            source = input_data
        else:
            raise ValueError("Inappropriate padding mode.")
        
        # Height indexer
        indh = np.arange(k) + np.expand_dims(s * np.arange(h_out), axis=1).repeat(k, axis=1)
        indh = indh.repeat(w_out, axis=0)
        indh = np.expand_dims(indh, axis=2)

        # Width indexer
        indw = np.arange(k) + np.expand_dims(s * np.arange(w_out), axis=1).repeat(k, axis=1)
        indw = np.tile(indw, reps=(h_out, 1))
        indw = np.expand_dims(indw, axis=1)

        # Extract patches; keep batch and channels
        patches = source[:, :, indh, indw]

        # Apply convolution
        conv = np.tensordot(patches, self.kernel, 
                            axes=([1, 3, 4], [2, 0, 1])).reshape(b, c_out, h_out, w_out)

        # Apply bias
        conv += np.expand_dims(self.bias, axis=(0, 2, 3))

        return conv

    def grad_x(self):
        pass
    
    def grad_kernel(self):
        pass

In [9]:
class Conv2DTrLayer(Layer):
    def __init__(self, kernel_size=3, input_channels=2, output_channels=3, 
                 padding=0, stride=1, K_init=None, b_init=None):      
        # padding: число (сколько отрезать от модифицированной входной карты)
        # Работаем с квадратными ядрами, поэтому kernel_size - одно число
        # stride - одно число (коэффициент расширения)
        # Фильтр размерности [kernel_size, kernel_size, input_channels, output_channels]
        self.name = 'Conv2DTr'
        self.kernel_size = kernel_size
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.kernel = K_init
        self.bias = b_init
        self.padding = padding
        self.stride = stride
    def forward(self, input_data):
        # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
        # Вначале нужно проверить на согласование размерностей входных данных и ядра!
        # Нужно заполнить Numpy-тензор out 
        out = np.empty([])
        return out

    def forward(self, input_data):
        pass
    def grad_x(self):
        pass
    def grad_kernel(self):
        pass

#### 1.4 Теперь настало время теста. 
#### Если вы всё сделали правильно, то запустив следующие ячейки у вас должна появиться надпись: Test PASSED

Переходить к дальнейшим заданиям не имеем никакого смысла, пока вы не добьётесь прохождение теста
    

#### Чтение данных

In [10]:
import numpy as np
from keras.utils import np_utils
from keras.datasets import mnist

np.random.seed(123)
 
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32') / 255
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32') / 255

Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

Using TensorFlow backend.
C:\Users\Dan\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Dan\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Dan\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])

(60000, 1, 28, 28) (60000, 10) (10000, 1, 28, 28) (10000, 10)


#### Подготовка моделей

In [11]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Convolution2D, Conv2D, MaxPooling2D
from keras.optimizers import Adam, SGD

print(keras.__version__)

def get_keras_model():
    input_image = Input(shape=(1, 28, 28))
    flatten = Flatten()(input_image)
    dense = Dense(10, activation='softmax')(flatten)
    model = Model(inputs=input_image, outputs=dense)

    sgd = SGD(lr=0.01, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])

    history = model.fit(X_train, Y_train, validation_split=0.25, 
                        batch_size=32, nb_epoch=2, verbose=1)
    return model

2.2.4


In [12]:
def get_our_model(keras_model):
    W_keras, b_keras = keras_model.get_weights()
    flatten = FlattenLayer()
    dense = DenseLayer(784, 10, W_init=W_keras, b_init=b_keras)
    softmax = Softmax()
    return Network([flatten, dense, softmax])

In [13]:
keras_model = get_keras_model()
our_model = get_our_model(keras_model)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 45000 samples, validate on 15000 samples
Epoch 1/2


C:\Users\Dan\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:21: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


45000/45000 [==============================] - 1s 31us/step - loss: 0.4483 - acc: 0.8756 - val_loss: 0.3339 - val_acc: 0.9081
Epoch 2/2
45000/45000 [==============================] - 1s 27us/step - loss: 0.3253 - acc: 0.9076 - val_loss: 0.3074 - val_acc: 0.9143


In [14]:
keras_pred = keras_model.predict(X_test)
our_pred = our_model.predict(X_test)
if np.sum(np.abs(keras_pred - our_pred)) < 0.01:
    print('Test PASSED')
else:
    print('Something went wrong!')

Test PASSED


### 2. Вычисление производных по входу для слоёв нейронной сети

В данном задании запрещено использовать численные формулы для вычисления производных.

#### 2.1  Реализуйте метод forward для класса CrossEntropy
Напоминание: $$ crossentropy = L(p, y) =  - \sum\limits_i y_i log p_i, $$
где вектор $(p_1, ..., p_k) $ -  выход классификационного алгоритма, а $(y_1,..., y_k)$ - правильные метки класса в унарной кодировке (one-hot encoding)

In [15]:
class CrossEntropy(object):
    def __init__(self):
        self.name = 'CrossEntropy'
    
    def forward(self, input_data, labels):
        return -np.sum(labels * np.log(input_data), axis=1)
    
    def grad_x(self, input_data, labels):
        return -labels / input_data

#### 2.2  Реализуйте метод grad_x класса CrossEntropy, который возвращает $\frac{\partial L}{\partial p}$

Проверить работоспособность кода поможет следующий тест:

In [16]:
def numerical_diff_loss(loss, x, labels):
    eps = 0.00001
    right_answer = []
    for i in range(len(x[0])):
        delta = np.zeros(len(x[0]))
        delta[i] = eps
        diff = (loss.forward(x + delta, labels) - loss.forward(x - delta, labels)) / (2 * eps)
        right_answer.append(diff)
    return np.array(right_answer).T

def test_loss(loss):
    x = np.array([[0.3, 0.2, 0.5], [0.3, 0.2, 0.5]])
    labels = np.array([[1, 2, 3], [2, 3, 4]])
    
    num_grad = numerical_diff_loss(loss, x, labels)
    grad = loss.grad_x(x, labels)
    if np.sum(np.abs(num_grad - grad)) < 0.01:
        print('Test PASSED')
    else:
        print('Something went wrong!')
        print('Numerical grad is')
        print(num_grad)
        print('Your grad is')
        print(grad)

In [17]:
loss = CrossEntropy()
test_loss(loss)

Test PASSED


#### 2.3  Реализуйте метод grad_x класса Softmax, который возвращает $\frac{\partial Softmax}{\partial x}$

Проверить работоспособность кода поможет следующий тест:

In [18]:
def numerical_diff_layer(layer, x):
    eps = 0.00001
    right_answer = []
    for i in range(len(x[0])):
        delta = np.zeros(len(x[0]))
        delta[i] = eps
        diff = (layer.forward(x + delta) - layer.forward(x - delta)) / (2 * eps)
        right_answer.append(diff.T)
    return np.array(right_answer).T

def test_layer(layer):
    x = np.array([[1, 2, 3], [2, -3, 4]])
    
    num_grad = numerical_diff_layer(layer, x)
    grad = layer.grad_x(x)
    if np.sum(np.abs(num_grad - grad)) < 0.01:
        print('Test PASSED')
    else:
        print('Something went wrong!')
        print('Numerical grad is')
        print(num_grad)
        print('Your grad is')
        print(grad)

In [19]:
layer = Softmax()
test_layer(layer)

Test PASSED


#### 2.4  Реализуйте метод grad_x для классов ReLU и DenseLayer

In [20]:
layer = ReLU()
test_layer(layer)

Test PASSED


In [21]:
layer = DenseLayer(3, 4)
test_layer(layer)

Test PASSED


#### 2.5 (4 балла) Для класса Network реализуйте метод grad_x, который должен реализовывать взятие производной от лосса по входу

In [22]:
def numerical_diff_net(net, x, labels):
    eps = 0.00001
    right_answer = []
    for i in range(len(x[0])):
        delta = np.zeros(len(x[0]))
        delta[i] = eps
        diff = (net.calculate_loss(x + delta, labels) - net.calculate_loss(x - delta, labels)) / (2 * eps)
        right_answer.append(diff)
    return np.array(right_answer).T

def test_net(net):
    x = np.array([[0.3, 0.2, 0.5], [0.3, 0.2, 0.5]])
    labels = np.array([[1, 2, 3], [2, 3, 4]])
    
    num_grad = numerical_diff_net(net, x, labels)
    grad = net.grad_x(x, labels)
    if np.sum(np.abs(num_grad - grad)) < 0.01:
        print('Test PASSED')
    else:
        print('Something went wrong!')
        print('Numerical grad is')
        print(num_grad)
        print('Your grad is')
        print(grad)

In [23]:
net = Network([DenseLayer(3, 10), ReLU(), DenseLayer(10, 3), Softmax()], loss=CrossEntropy())
test_net(net)

Test PASSED


### 3. Реализация градиентов по параметрам и метода обратного распространения ошибки с обновлением парметров сети

#### 3.1  Реализуйте функции grad_b и grad_W. При подготовке теста grad_W предполагается, что W является отномерным вектором.

In [24]:
def numerical_grad_b(input_size, output_size, W, b, x):
    eps = 0.00001
    right_answer = []
    for i in range(len(b)):
        delta = np.zeros(b.shape)
        delta[i] = eps
        dense1 = DenseLayer(input_size, output_size, W_init=W, b_init=b+delta)
        dense2 = DenseLayer(input_size, output_size, W_init=W, b_init=b-delta)
        diff = (dense1.forward(x) - dense2.forward(x)) / (2 * eps)
        right_answer.append(diff.T)
    return np.array(right_answer).T

def test_grad_b():
    input_size, output_size = 3, 4 
    W_init = np.random.random((input_size, output_size))
    b_init = np.random.random((output_size,))
    x = np.random.random((2, input_size))

    num_grad = numerical_grad_b(input_size, output_size, W_init, b_init, x)
    dense = DenseLayer(input_size, output_size, W_init, b_init)
    grad = dense.grad_b(x)
    if np.sum(np.abs(num_grad - grad)) < 0.01:
        print('Test PASSED')
    else:
        print('Something went wrong!')
        print('Numerical grad is')
        print(num_grad)
        print('Your grad is')
        print(grad)

In [25]:
test_grad_b()

Test PASSED


In [26]:
def numerical_grad_W(input_size, output_size, W, b, x):
    eps = 0.00001
    right_answer = []
    for i in range(W.shape[0]):
        for j in range(W.shape[1]):
            delta = np.zeros(W.shape)
            delta[i, j] = eps
            dense1 = DenseLayer(input_size, output_size, W_init=W+delta, b_init=b)
            dense2 = DenseLayer(input_size, output_size, W_init=W-delta, b_init=b)
            diff = (dense1.forward(x) - dense2.forward(x)) / (2*eps)
            right_answer.append(diff.T)
    return np.array(right_answer).T

def test_grad_W():
    input_size, output_size = 3, 4
    W_init = np.random.random((input_size, output_size))
    b_init = np.random.random((4,))
    x = np.random.random((2, input_size))
    
    num_grad = numerical_grad_W(input_size, output_size, W_init, b_init, x)
    dense = DenseLayer(input_size, output_size, W_init, b_init)
    grad = dense.grad_W(x)
    if np.sum(np.abs(num_grad - grad)) < 0.01:
        print('Test PASSED')
    else:
        print('Something went wrong!')
        print('Numerical grad is')
        print(num_grad)
        print('Your grad is')
        print(grad)

In [27]:
test_grad_W()

Test PASSED


#### 3.2 Полностью реализуйте метод обратного распространения ошибки в функции train_step класса Network


Рекомендуем реализовать сначала функцию Network.grad_param(), которая возвращает список длиной в количество слоёв и элементом которого является список градиентов по параметрам.
После чего, имея список градиентов, написать функцию обновления параметров для каждого слоя. 

Совет: рекомендуем написать тест для кода подсчета градиента по параметрам, чтобы быть уверенным в том, что градиент через всю сеть считается правильно
    

In [28]:
def numerical_grad_param(net, x, labels):
    eps = 0.00001
    right_answer = []
    W = net.layers[0].W
    for i in range(W.shape[0]):
        for j in range(W.shape[1]):
            delta = np.zeros(W.shape)
            delta[i, j] = eps
            layer1 = DenseLayer(W.shape[0], W.shape[1], W_init=W+delta)
            layer2 = DenseLayer(W.shape[0], W.shape[1], W_init=W-delta)
            net1 = Network([layer1] + net.layers[1:], loss=net.loss)
            net2 = Network([layer2] + net.layers[1:], loss=net.loss)
            diff = (net1.calculate_loss(x, labels) - net2.calculate_loss(x, labels)) / (2 * eps)
            right_answer.append(diff)
    return np.array(right_answer).T.reshape(x.shape[0], W.shape[0], W.shape[1])

def test_grad_param():
    net = Network([DenseLayer(768, 20), ReLU(), DenseLayer(20, 3), Softmax()], loss=CrossEntropy())
    x = np.random.random((2, 768))
    labels = np.array([[1, 2, 3], [2, 3, 4]])

    num_grad = numerical_grad_param(net, x, labels)
    grad = net.grad_param(x, labels)[0][0].reshape((x.shape[0],) + net.layers[0].W.shape)
    if np.sum(np.abs(num_grad - grad)) < 0.01:
        print('Test PASSED')
    else:
        print('Something went wrong!')
        print('Numerical grad is')
        print(num_grad)
        print('Your grad is')
        print(grad)

In [29]:
test_grad_param()

Test PASSED


#### 3.3 Ознакомьтесь с реализацией функции fit класса Network. Запустите обучение модели. Если всё работает правильно, то точность на валидации должна будет возрастать

In [30]:
net = Network([DenseLayer(784, 10), Softmax()], loss=CrossEntropy())
trainX = X_train.reshape(len(X_train), -1)
net.fit(trainX[::3], Y_train[::3], validation_split=0.25, 
        batch_size=16, nb_epoch=5, learning_rate=0.01)

100%|██████████| 937/937 [00:13<00:00, 70.82it/s]


1 epoch: val 0.85


100%|██████████| 937/937 [00:13<00:00, 71.10it/s]


2 epoch: val 0.87


100%|██████████| 937/937 [00:13<00:00, 70.62it/s]


3 epoch: val 0.88


100%|██████████| 937/937 [00:13<00:00, 71.05it/s]


4 epoch: val 0.89


100%|██████████| 937/937 [00:13<00:00, 69.89it/s]


5 epoch: val 0.89


In [31]:
net = Network([DenseLayer(784, 20), ReLU(), DenseLayer(20, 10), Softmax()], loss=CrossEntropy())
trainX = X_train.reshape(len(X_train), -1)
net.fit(trainX[::6], Y_train[::6], validation_split=0.25, 
        batch_size=16, nb_epoch=5, learning_rate=0.001)

100%|██████████| 468/468 [00:27<00:00, 17.02it/s]


1 epoch: val 0.19


100%|██████████| 468/468 [00:27<00:00, 17.02it/s]


2 epoch: val 0.35


100%|██████████| 468/468 [00:27<00:00, 17.11it/s]


3 epoch: val 0.44


100%|██████████| 468/468 [00:27<00:00, 17.13it/s]


4 epoch: val 0.52


100%|██████████| 468/468 [00:27<00:00, 17.10it/s]

5 epoch: val 0.59


#### 3.5 Продемонстрируйте, что ваша реализация позволяет обучать более глубокие нейронные сети 